# FinnGen - Data Engineering Notebook - Causal

## Data Descriptions

### gwas


The `{endpoint}.gz` file has the following structure:

| Column name   | Description                                                 |
| ------------- | ----------------------------------------------------------- |
| #chrom        | chromosome on build GRCh38 (1-23)                           |
| pos           | position in base pairs on build GRCh38                       |
| ref           | reference allele                                            |
| alt           | alternative allele (effect allele)                           |
| rsids         | variant identifier                                          |
| nearest_genes | nearest gene(s) (comma separated) from variant               |
| pval          | p-value from [source]                                        |
| mlogp         | -log10(p-value)                                             |
| beta          | effect size (log(OR) scale) estimated with [source]          |
| sebeta        | standard error of effect size estimated with [source]        |
| af_alt        | alternative (effect) allele frequency                        |
| af_alt_cases  | alternative (effect) allele frequency among cases            |
| af_alt_controls | alternative (effect) allele frequency among controls         |


### finemap

{endpoint}.SUSIE.snp.bgz` contains variant summaries with credible set information and has the following structure:

| Column name    | Description                                                        |
| -------------- | ------------------------------------------------------------------ |
| trait          | endpoint name                                                      |
| region         | chr:start-end                                                      |
| v              | variant identifier                                                 |
| rsid           | rs variant identifier                                              |
| chromosome     | chromosome on build GRCh38 (1-22, X)                                |
| position       | position in base pairs on build GRCh38                              |
| allele1        | reference allele                                                   |
| allele2        | alternative allele (effect allele)                                  |
| maf            | minor allele frequency                                             |
| beta           | effect size GWAS                                                   |
| se             | standard error GWAS                                                |
| p              | p-value GWAS                                                       |
| mean           | posterior expectation of true effect size                           |
| sd             | posterior standard deviation of true effect size                   |
| prob           | posterior probability of association                                |
| cs             | identifier of 95% credible set (-1 = variant is not part of credible set) |
| lead_r2        | r2 value to a lead variant (the one with maximum PIP) in a credible set |
| alphax         | posterior inclusion probability for the x-th single effect (x := 1..L where L is the number of single effects (causal variants) specified; default: L = 10) |

## Libraries

In [1]:
import sys
import pandas as pd
import numpy as np
import requests
import time
from concurrent.futures import ThreadPoolExecutor



print("Python version:", sys.version)
print("Pandas version:", pd.__version__)
print("NumPy version:", np.__version__)

Python version: 3.11.4 (tags/v3.11.4:d2340ef, Jun  7 2023, 05:45:37) [MSC v.1934 64 bit (AMD64)]
Pandas version: 1.5.3
NumPy version: 1.24.1


## Import data

In [2]:
import os

# Get the current working directory
current_directory = os.getcwd()

print(current_directory)

C:\Users\Windows\Desktop\GeoGWAS\FinnGen\notebooks\precausal


In [3]:
# Read the 'finemap' file into a pandas DataFrame
finemap = pd.read_csv('C:/Users/Windows/Desktop/GeoGWAS/FinnGen/data/finemapping_summary_finngen_R9_I9_HYPTENS.SUSIE_99.snp.filter.tsv', low_memory=False, sep='\t')


# Read the 'causal' file into a pandas DataFrame
precausal = pd.read_csv('C:/Users/Windows/Desktop/GeoGWAS/FinnGen/data/finemapping_summary_finngen_R9_I9_HYPTENS.SUSIE.cred.summary.tsv', low_memory=False, sep='\t')


# Read the 'causal' file into a pandas DataFrame
causal = pd.read_csv('C:/Users/Windows/Desktop/GeoGWAS/FinnGen/data/causal-hyp.tsv', low_memory=False, sep='\t')


# Read the 'gwas' file into a pandas DataFrame
gwas = pd.read_csv('C:/Users/Windows/Desktop/GeoGWAS/FinnGen/data/summary_stats_finngen_R9_I9_HYPTENS.tsv', low_memory=False, sep='\t')

In [4]:
print("NaNs and missing values in 'gwas':")
empty = gwas.isna().sum()
print(empty)

NaNs and missing values in 'gwas':
#chrom                   0
pos                      0
ref                      0
alt                      0
rsids              1366441
nearest_genes       727861
pval                     0
mlogp                    0
beta                     0
sebeta                   0
af_alt                   0
af_alt_cases             0
af_alt_controls          0
dtype: int64


## Explore data

In [5]:
gwas

,#chrom,pos,ref,alt,rsids,nearest_genes,pval,mlogp,beta,sebeta,af_alt,af_alt_cases,af_alt_controls
0,1,13668,G,A,rs2691328,OR4F5,0.106658,0.972006,-0.114822,0.071168,0.005846,0.005683,0.005914
1,1,14773,C,T,rs878915777,OR4F5,0.620115,0.207528,-0.021548,0.043470,0.013501,0.013448,0.013524
2,1,15585,G,A,rs533630043,OR4F5,0.859628,0.065689,-0.023716,0.134105,0.001112,0.001117,0.001109
3,1,16549,T,C,rs1262014613,OR4F5,0.321844,0.492355,-0.215787,0.217818,0.000563,0.000556,0.000566
4,1,16567,G,C,rs1194064194,OR4F5,0.764225,0.116779,0.021523,0.071757,0.004192,0.004207,0.004186
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20170229,23,155697920,G,A,NaN,NaN,0.435606,0.360906,0.004066,0.005215,0.291210,0.291159,0.291231
20170230,23,155698443,C,A,NaN,NaN,0.559723,0.252027,0.025926,0.044450,0.003263,0.003298,0.003248
20170231,23,155698490,C,T,NaN,NaN,0.007623,2.117900,-0.043135,0.016165,0.024340,0.023984,0.024489
20170232,23,155699751,C,T,NaN,NaN,0.160090,0.795637,0.007715,0.005492,0.245151,0.245738,0.244904


In [6]:
finemap.head()

,trait,region,v,cs,cs_specific_prob,chromosome,position,allele1,allele2,maf,beta,p,se,most_severe,gene_most_severe
0,I9_HYPTENS,chr1:5120847-8120847,1:6584663:G:A,1,0.043208,chr1,6584663,G,A,0.281248,-0.038346,2.351200e-09,0.006421,intron_variant,ZBTB48
1,I9_HYPTENS,chr1:5120847-8120847,1:6585807:G:A,1,0.044924,chr1,6585807,G,A,0.281268,-0.038387,2.255640e-09,0.006421,non_coding_transcript_exon_variant,ZBTB48
2,I9_HYPTENS,chr1:5120847-8120847,1:6594993:G:A,1,0.057924,chr1,6594993,G,A,0.280470,-0.038723,1.677950e-09,0.006426,3_prime_UTR_variant,KLHL21
3,I9_HYPTENS,chr1:5120847-8120847,1:6599445:G:A,1,0.078622,chr1,6599445,G,A,0.281519,-0.039011,1.202020e-09,0.006416,synonymous_variant,KLHL21
4,I9_HYPTENS,chr1:5120847-8120847,1:6600289:A:G,1,0.075465,chr1,6600289,A,G,0.281502,-0.038967,1.255360e-09,0.006416,intron_variant,KLHL21


In [7]:
precausal.head()

,trait,region,cs,cs_log10bf,cs_avg_r2,cs_min_r2,low_purity,cs_size,good_cs,cs_id,v,rsid,p,beta,sd,prob,cs_specific_prob,most_severe,gene_most_severe
0,I9_HYPTENS,chr11:365986-3365986,3,1.476225,0.317892,0.030052,True,11,False,chr11:365986-3365986_3,11:1899025:T:G,chr11_1899025_T_G,2.560350e-19,0.061645,0.005078,0.030545,0.021575,intron_variant,LINC01150
1,I9_HYPTENS,chr1:54982752-57982752,2,1.688606,0.267826,0.007267,True,61,False,chr1:54982752-57982752_2,1:57780626:A:G,chr1_57780626_A_G,4.735000e-08,-0.031565,0.011374,0.169756,0.169537,intron_variant,DAB1
2,I9_HYPTENS,chr1:54982752-57982752,1,5.047026,0.882801,0.730739,False,15,True,chr1:54982752-57982752_1,1:56482752:T:C,chr1_56482752_T_C,1.398940e-11,0.040267,0.012328,0.113580,0.113364,intron_variant,AC119674.2
3,I9_HYPTENS,chr16:80069630-83069630,2,1.799040,0.364302,0.048222,True,38,False,chr16:80069630-83069630_2,16:81476866:C:G,chr16_81476866_C_G,4.057890e-10,-0.041016,0.007396,0.051703,0.050699,intron_variant,CMIP
4,I9_HYPTENS,chr13:20244714-23244714,1,5.553981,0.932232,0.871865,False,14,True,chr13:20244714-23244714_1,13:21744714:A:G,chr13_21744714_A_G,1.170040e-11,-0.040128,0.011729,0.096990,0.096802,intergenic_variant,NaN


In [8]:
causal.head()

,association_info.ancestry,association_info.doi,association_info.gwas_catalog_id,association_info.neg_log_pval,association_info.otg_id,association_info.pubmed_id,association_info.url,gold_standard_info.evidence.class,gold_standard_info.evidence.confidence,gold_standard_info.evidence.curated_by,...,sentinel_variant.alleles.alternative,sentinel_variant.alleles.reference,sentinel_variant.locus_GRCh37.chromosome,sentinel_variant.locus_GRCh37.position,sentinel_variant.locus_GRCh38.chromosome,sentinel_variant.locus_GRCh38.position,sentinel_variant.rsid,trait_info.ontology,trait_info.reported_trait_name,trait_info.standard_trait_name
0,European=408343,10.1038/s41588-018-0184-y,NaN,14.397940,SAIGE_401,NaN,NaN,drug,High,Ed Mountjoy,...,C,G,10,115805056,10,114045297,NaN,EFO_0000537,Hypertension,hypertension
1,European=408343,10.1038/s41588-018-0184-y,NaN,15.000000,SAIGE_401,NaN,NaN,drug,Low,Ed Mountjoy,...,T,A,11,1873232,11,1852002,NaN,EFO_0000537,Hypertension,hypertension
2,European=408343,10.1038/s41588-018-0184-y,NaN,19.619789,SAIGE_401,NaN,NaN,drug,Low,Ed Mountjoy,...,T,C,11,1888614,11,1867384,NaN,EFO_0000537,Hypertension,hypertension
3,European=69395,NaN,GCST001238,10.301030,GCST001238,21909115.0,NaN,drug,Low,Ed Mountjoy,...,C,G,11,100593538,11,100722807,NaN,EFO_0000537,Hypertension,hypertension
4,European=408343,10.1038/s41588-018-0184-y,NaN,20.274905,SAIGE_401,NaN,NaN,drug,Low,Ed Mountjoy,...,C,T,11,100610546,11,100739815,NaN,EFO_0000537,Hypertension,hypertension


In [9]:
def explore_dataframe(dataframe, dataframe_name):
    print("=== DataFrame Exploration: {} ===".format(dataframe_name))
    print("Number of Rows: {}".format(dataframe.shape[0]))
    print("Number of Columns: {}".format(dataframe.shape[1]))
    print("Column Names: {}".format(", ".join(dataframe.columns)))
    print("\nData Types of Columns:")
    print(dataframe.dtypes)
    print("\nNull Value Counts:")
    print(dataframe.isnull().sum())
    print("\nSummary Statistics:")
    print(dataframe.describe())
    print("=== End of DataFrame Exploration: {} ===\n".format(dataframe_name))
    
#explore_dataframe(gwas, "gwas")
#explore_dataframe(causal, "causal")
#explore_dataframe(finemap, "finemap")

## Data manipulation

### Adjust `chromosome` in `finemap`

In [10]:
# Extract number from 'chromosome' and replace 'X' with '23'
finemap['chromosome'] = finemap['chromosome'].str.extract('(\d+|X)', expand=False).replace('X', '23')

# Convert 'chromosome' column to 'int64'
finemap['chromosome'] = finemap['chromosome'].astype('int64')

# Assertions to verify the data manipulations
assert finemap['chromosome'].dtype == 'int64'  
assert finemap['chromosome'].isin(range(1, 24)).all()  

### Adjust `v` in `finemap`

In [11]:
# Replace 'X' with '23' in 'v' column of finemap
finemap['v'] = finemap['v'].str.replace(r'(^X:)', '23:', regex=True)

# Assert 'X' is not in 'v' column anymore
assert 'X' not in finemap['v']

### Create `finemapped` in `gwas`

In [12]:
# Create the 'id' column in the 'gwas' DataFrame
gwas['id'] = gwas['#chrom'].astype(str) + ':' + gwas['pos'].astype(str) + ':' + gwas['ref'] + ':' + gwas['alt']

# Create a set for faster lookup
finemap_set = set(finemap['v'].values)

# Use the set for lookup
gwas['finemapped'] = gwas['id'].apply(lambda x: 1 if x in finemap_set else 0)

# Count the number of 1s in the 'finemapped' column
count_ones = gwas['finemapped'].sum()

# Perform assertions to validate the results
assert len(gwas) == len(gwas['id']) == len(gwas['finemapped']), "Lengths do not match."
assert count_ones <= len(gwas), "Invalid count of 1s."

print("Assertions passed successfully.")

Assertions passed successfully.


### Create `precausal` in `gwas`

In [13]:
precausal_snp_set = set(precausal['v'])

gwas['precausal'] = gwas['id'].apply(lambda x: 1 if x in precausal_snp_set else 0)
print(gwas['precausal'].sum())

288


### Create `causal` in `gwas`

In [14]:
# create a combined column in both dataframes
causal['combined'] = causal['sentinel_variant.locus_GRCh38.chromosome'].astype(str) + '_' + causal['sentinel_variant.locus_GRCh38.position'].astype(str)
gwas['combined'] = gwas['#chrom'].astype(str) + '_' + gwas['pos'].astype(str)

# create a set from the 'combined' column in causal
causal_combined_set = set(causal['combined'])

# create a new column 'causal' in gwas that checks if 'combined' is in causal_combined_set
gwas['causal'] = gwas['combined'].apply(lambda x: 1 if x in causal_combined_set else 0)

print(gwas['causal'].sum())

33


### Extract `trait` from `finemap` to `gwas`

In [15]:
unique_trait = finemap['trait'].unique()
trait_string = unique_trait[0]
gwas['trait'] = trait_string

## Export csv

In [16]:
finemapped_gwas = gwas[(gwas['finemapped'] == 1) | (gwas['precausal'] == 1)]

In [17]:
print(finemapped_gwas['precausal'].sum())

288


In [18]:
finemapped_gwas = finemapped_gwas.drop(['causal', 'finemapped', 'combined'], axis=1)

In [19]:
finemapped_gwas

,#chrom,pos,ref,alt,rsids,nearest_genes,pval,mlogp,beta,sebeta,af_alt,af_alt_cases,af_alt_controls,id,precausal,trait
23344,1,3412095,C,T,rs2493292,PRDM16,8.673610e-11,10.06180,0.059679,0.009198,0.109426,0.112670,0.108063,1:3412095:C:T,1,I9_HYPTENS
50476,1,6584663,G,A,rs731024,ZBTB48,2.351200e-09,8.62871,-0.038346,0.006421,0.281248,0.277440,0.282847,1:6584663:G:A,0,I9_HYPTENS
50484,1,6585807,G,A,rs6682920,ZBTB48,2.255640e-09,8.64673,-0.038387,0.006421,0.281268,0.277461,0.282867,1:6585807:G:A,0,I9_HYPTENS
50547,1,6594993,G,A,rs2235566,KLHL21,1.677950e-09,8.77522,-0.038723,0.006426,0.280470,0.276618,0.282088,1:6594993:G:A,0,I9_HYPTENS
50597,1,6599445,G,A,rs2232460,KLHL21,1.202020e-09,8.92009,-0.039011,0.006416,0.281519,0.277570,0.283177,1:6599445:G:A,0,I9_HYPTENS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19925078,23,103566142,C,T,NaN,NaN,2.266050e-08,7.64473,0.090374,0.016166,0.021548,0.023120,0.020887,23:103566142:C:T,0,I9_HYPTENS
19925210,23,103613602,G,A,NaN,NaN,2.810740e-08,7.55118,0.089673,0.016149,0.021587,0.023138,0.020935,23:103613602:G:A,0,I9_HYPTENS
19925235,23,103622280,GTC,G,NaN,NaN,2.325630e-08,7.63346,0.090112,0.016132,0.021632,0.023192,0.020976,23:103622280:GTC:G,0,I9_HYPTENS
19925467,23,103686440,CT,C,NaN,NaN,8.519420e-08,7.06959,0.098822,0.018451,0.016685,0.017929,0.016162,23:103686440:CT:C,0,I9_HYPTENS


In [20]:
#finemapped_gwas.to_csv('hyp-finemapped-precausal.csv', index=False)